In [13]:
import pandas as pd

# Load the HPI CSV file into a DataFrame
df = pd.read_csv("Home Price Index Recent Download\hpi_master.csv")
print("Original DataFrame shape:", df.shape)
df.head()


Original DataFrame shape: (128142, 10)


,hpi_type,hpi_flavor,frequency,level,place_name,place_id,yr,period,index_nsa,index_sa
0,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,1991,1,100.00,100.00
1,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,1991,2,100.90,100.94
2,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,1991,3,101.29,100.90
3,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,1991,4,101.69,100.97
4,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,1991,5,102.31,101.32


In [14]:
# Set Average Interest Rates based on the freddiemac PMMS website (https://www.freddiemac.com/pmms)
rates_df = pd.read_excel("Home Price Index Recent Download\historicalweeklydata.xlsx",skiprows=6)
num_rows = rates_df.shape[0]
average_interest_rate = rates_df.iloc[num_rows-2,1]
print("Latest Average 30 year mortage interest rate is:", str(average_interest_rate)+"%")


Latest Average 30 year mortage interest rate is: 6.63%


In [15]:
# Load the Zillow CSV, keeping only RegionName and "1/31/2025"
df_zillow = pd.read_csv("Home Price Index Recent Download\State_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv", usecols=["RegionName", "2025-01-31"])

# Rename columns:
#   "1/31/2025" to "Typical Home Value as of 01/31/2024"
#   "RegionName" to "State_merge" so the original "State" column in inflation_pivot remains intact
df_zillow.rename(columns={
    "2025-01-31": "Typical Home Value as of 01/31/2024",
    "RegionName": "State_merge"
}, inplace=True)

#Data showing hoe values for latest month
display(df_zillow.head())

,State_merge,Typical Home Value as of 01/31/2024
0,California,773347.013436
1,Texas,298510.934462
2,Florida,385540.771612
3,New York,477332.960271
4,Pennsylvania,265021.074573


In [16]:
# Filter the HPI DataFrame to include rows where:
#    - level == "State"
#    - hpi_type == "traditional"
#    - hpi_flavor == "all-transactions"
#    - yr >= 2019 and yr <= 2024 (so we have 2019 Q4 baseline for Q1 2020 inflation)
df_filtered = df[
    (df["level"] == "State") &
    (df["hpi_type"] == "traditional") &
    (df["hpi_flavor"] == "all-transactions") &
    (df["yr"] >= 2019) &
    (df["yr"] <= 2024)
]

# Rename columns: "place_name" -> "State" and "period" -> "qtr"
df_filtered.rename(columns={"place_name": "State", "period": "qtr"}, inplace=True)

print("Filtered DataFrame shape:", df_filtered.shape)
df_filtered.head()


Filtered DataFrame shape: (1224, 10)


C:\Users\16823\AppData\Local\Temp\ipykernel_26936\2859956643.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.rename(columns={"place_name": "State", "period": "qtr"}, inplace=True)


,hpi_type,hpi_flavor,frequency,level,State,place_id,yr,qtr,index_nsa,index_sa
71619,traditional,all-transactions,quarterly,State,Alaska,AK,2019,1,324.20,NaN
71620,traditional,all-transactions,quarterly,State,Alaska,AK,2019,2,328.23,NaN
71621,traditional,all-transactions,quarterly,State,Alaska,AK,2019,3,330.40,NaN
71622,traditional,all-transactions,quarterly,State,Alaska,AK,2019,4,330.12,NaN
71623,traditional,all-transactions,quarterly,State,Alaska,AK,2020,1,334.08,NaN


In [17]:
# Remove the column "index_sa"
df_filtered = df_filtered.drop(columns=["index_sa"], errors="ignore")

# Create a combined "year_quarter" column
df_filtered["year_quarter"] = df_filtered["yr"].astype(str) + "-Q" + df_filtered["qtr"].astype(str)

# Pivot the DataFrame so that:
#    - Rows: State
#    - Columns: year_quarter (e.g., 2020-Q1, 2020-Q2, etc.)
#    - Values: index_nsa
df_pivot = df_filtered.pivot(index="State", columns="year_quarter", values="index_nsa")

df_pivot.head()



year_quarter,2019-Q1,2019-Q2,2019-Q3,2019-Q4,2020-Q1,2020-Q2,2020-Q3,2020-Q4,2021-Q1,2021-Q2,...,2022-Q3,2022-Q4,2023-Q1,2023-Q2,2023-Q3,2023-Q4,2024-Q1,2024-Q2,2024-Q3,2024-Q4
State,,,,,,,,,,,,,,,,,,,,,
Alabama,322.18,326.71,331.46,335.36,339.24,344.14,350.74,358.16,365.59,382.64,...,470.85,471.38,476.51,485.75,496.88,494.24,502.48,515.15,521.27,526.58
Alaska,324.20,328.23,330.40,330.12,334.08,337.16,341.45,344.32,348.06,361.06,...,420.30,414.18,425.83,437.79,439.08,435.76,444.06,460.94,466.19,463.78
Arizona,412.55,420.77,427.10,434.47,443.17,450.93,461.94,478.24,499.55,539.36,...,700.96,676.54,671.61,682.99,697.85,705.54,708.88,722.09,728.74,727.74
Arkansas,286.49,289.85,293.14,295.54,297.31,299.99,304.47,311.94,317.66,333.49,...,414.80,414.39,421.51,431.63,439.30,443.03,445.94,456.00,464.28,465.56
California,645.68,652.26,659.16,663.45,669.69,677.37,685.31,699.46,719.04,761.72,...,915.45,891.69,888.87,911.51,923.45,926.65,941.18,956.39,965.29,968.88


In [18]:
# Calculate quarter-over-quarter inflation for Q1 2020 to Q4 2024
# Inflation formula for a given quarter col:
# inflation = (index_nsa_current_quarter / index_nsa_previous_quarter) * 100
# Create a new DataFrame, inflation_pivot, with the same row index (State).

def get_previous_quarter(yq):
    """
    Given a string like '2020-Q1', return the previous quarter (e.g. '2019-Q4').
    """
    year_str, quarter_str = yq.split("-Q")
    year = int(year_str)
    quarter = int(quarter_str)
    
    if quarter == 1:
        year -= 1
        quarter = 4
    else:
        quarter -= 1
    
    return f"{year}-Q{quarter}"

# Build a list of quarters from 2020-Q1 to 2024-Q4
inflation_quarters = []
for yr in range(2020, 2025):  # up to 2024 inclusive
    for qtr in range(1, 5):
        # Stop if we go beyond 2024-Q4
        if yr == 2024 and qtr > 4:
            break
        inflation_quarters.append(f"{yr}-Q{qtr}")

# Create a new pivot table for inflation
inflation_pivot = pd.DataFrame(index=df_pivot.index, columns=inflation_quarters)

#So far we now have a blank data frame with rows as states and columns as the years from 2020 Q1 to 2024 Q4.
#This is also saved in a list called inflation_quarters


for col in inflation_quarters:
    prev_col = get_previous_quarter(col)
    # Calculate inflation only if both current and previous columns exist
    #This will start with comparing 2020 Q1 to 2019 Q4
    if col in df_pivot.columns and prev_col in df_pivot.columns:
        inflation_pivot[col] = ((df_pivot[col] / df_pivot[prev_col]) * 100) - 100
    else:
        # If the previous quarter is missing then blank
        inflation_pivot[col] = pd.NA

# Round to two decimals
inflation_pivot = inflation_pivot.round(2)
inflation_pivot.head()


,2020-Q1,2020-Q2,2020-Q3,2020-Q4,2021-Q1,2021-Q2,2021-Q3,2021-Q4,2022-Q1,2022-Q2,2022-Q3,2022-Q4,2023-Q1,2023-Q2,2023-Q3,2023-Q4,2024-Q1,2024-Q2,2024-Q3,2024-Q4
State,,,,,,,,,,,,,,,,,,,,
Alabama,1.16,1.44,1.92,2.12,2.07,4.66,5.68,4.09,2.86,5.67,2.92,0.11,1.09,1.94,2.29,-0.53,1.67,2.52,1.19,1.02
Alaska,1.20,0.92,1.27,0.84,1.09,3.73,4.54,1.45,2.63,5.76,1.12,-1.46,2.81,2.81,0.29,-0.76,1.90,3.80,1.14,-0.52
Arizona,2.00,1.75,2.44,3.53,4.46,7.97,8.30,5.10,4.78,7.53,1.34,-3.48,-0.73,1.69,2.18,1.10,0.47,1.86,0.92,-0.14
Arkansas,0.60,0.90,1.49,2.45,1.83,4.98,5.62,4.34,2.79,6.17,3.43,-0.10,1.72,2.40,1.78,0.85,0.66,2.26,1.82,0.28
California,0.94,1.15,1.17,2.06,2.80,5.94,5.21,4.05,4.07,6.14,-0.62,-2.60,-0.32,2.55,1.31,0.35,1.57,1.62,0.93,0.37


In [19]:
# Now I add two calculated columns to the inflation data to measure 2024 Inflation and overall cumulative inflation from 2020 to 2024

# Create "Inflation 2024" columns which is (2024 Q4 / 2023 Q4) * 100
if "2024-Q4" in df_pivot.columns and "2023-Q4" in df_pivot.columns:
    inflation_pivot["Inflation 2024 %"] = (
        (df_pivot["2024-Q4"] / df_pivot["2023-Q1"] * 100) - 100
    ).round(2)
else:
    inflation_pivot["Inflation 2024 %"] = pd.NA

# Create "Cumulative Inflation 2020-2024" column which is (2020 Q1 / 2024 Q4) * 100
if "2020-Q1" in df_pivot.columns and "2024-Q4" in df_pivot.columns:
    inflation_pivot["Cumulative Inflation 2020-2024 %"] = (
        (df_pivot["2024-Q4"] / df_pivot["2020-Q1"] * 100)-100
    ).round(2)
else:
    inflation_pivot["Cumulative Inflation 2020-2024 %"] = pd.NA

inflation_pivot.head()


,2020-Q1,2020-Q2,2020-Q3,2020-Q4,2021-Q1,2021-Q2,2021-Q3,2021-Q4,2022-Q1,2022-Q2,...,2023-Q1,2023-Q2,2023-Q3,2023-Q4,2024-Q1,2024-Q2,2024-Q3,2024-Q4,Inflation 2024 %,Cumulative Inflation 2020-2024 %
State,,,,,,,,,,,,,,,,,,,,,
Alabama,1.16,1.44,1.92,2.12,2.07,4.66,5.68,4.09,2.86,5.67,...,1.09,1.94,2.29,-0.53,1.67,2.52,1.19,1.02,10.51,55.22
Alaska,1.20,0.92,1.27,0.84,1.09,3.73,4.54,1.45,2.63,5.76,...,2.81,2.81,0.29,-0.76,1.90,3.80,1.14,-0.52,8.91,38.82
Arizona,2.00,1.75,2.44,3.53,4.46,7.97,8.30,5.10,4.78,7.53,...,-0.73,1.69,2.18,1.10,0.47,1.86,0.92,-0.14,8.36,64.21
Arkansas,0.60,0.90,1.49,2.45,1.83,4.98,5.62,4.34,2.79,6.17,...,1.72,2.40,1.78,0.85,0.66,2.26,1.82,0.28,10.45,56.59
California,0.94,1.15,1.17,2.06,2.80,5.94,5.21,4.05,4.07,6.14,...,-0.32,2.55,1.31,0.35,1.57,1.62,0.93,0.37,9.00,44.68


In [20]:
#Add columns from the Zillow file

# Before merging, convert the pivot index to a column named "State"
inflation_pivot = inflation_pivot.reset_index()

# Merge inflation_pivot with the Zillow data on the "State" column
df_merged = pd.merge(
    inflation_pivot,
    df_zillow,
    how="left",
    left_on="State",
    right_on="State_merge"
)

# Drop the extra "State_merge" column
df_merged.drop(columns=["State_merge"], inplace=True)

df_merged["Typical Home Value as of 01/31/2024"] = df_merged["Typical Home Value as of 01/31/2024"].round(2)

# Add the constant column for average interest rate (6.63)
df_merged["Average Interest Rate (30 Yr Mortgage)"] = average_interest_rate

print("After merge:")
display(df_merged.head())

# Update inflation_pivot with the merged results
inflation_pivot = df_merged

After merge:


,State,2020-Q1,2020-Q2,2020-Q3,2020-Q4,2021-Q1,2021-Q2,2021-Q3,2021-Q4,2022-Q1,...,2023-Q3,2023-Q4,2024-Q1,2024-Q2,2024-Q3,2024-Q4,Inflation 2024 %,Cumulative Inflation 2020-2024 %,Typical Home Value as of 01/31/2024,Average Interest Rate (30 Yr Mortgage)
0,Alabama,1.16,1.44,1.92,2.12,2.07,4.66,5.68,4.09,2.86,...,2.29,-0.53,1.67,2.52,1.19,1.02,10.51,55.22,224910.79,6.63
1,Alaska,1.20,0.92,1.27,0.84,1.09,3.73,4.54,1.45,2.63,...,0.29,-0.76,1.90,3.80,1.14,-0.52,8.91,38.82,366633.08,6.63
2,Arizona,2.00,1.75,2.44,3.53,4.46,7.97,8.30,5.10,4.78,...,2.18,1.10,0.47,1.86,0.92,-0.14,8.36,64.21,424841.80,6.63
3,Arkansas,0.60,0.90,1.49,2.45,1.83,4.98,5.62,4.34,2.79,...,1.78,0.85,0.66,2.26,1.82,0.28,10.45,56.59,208999.63,6.63
4,California,0.94,1.15,1.17,2.06,2.80,5.94,5.21,4.05,4.07,...,1.31,0.35,1.57,1.62,0.93,0.37,9.00,44.68,773347.01,6.63


In [21]:
# I now create 5 new columns all intended to guide the users decision to buy an investment home in a particlar state
# Column 1: Down Payment
# Column 2: Monthly loan payment
# Column 3: Average Rent in state
# Column 4: Potential Monthly Payoff
# Column 5: Average Rent Yield

# Ensure the Typical Home Value column is numeric and round it to two decimals
inflation_pivot["Typical Home Value as of 01/31/2024"] = pd.to_numeric(
    inflation_pivot["Typical Home Value as of 01/31/2024"], errors="coerce"
).round(2)

# Column 1: Down Payment = 20% of Typical Home Value
inflation_pivot["Down Payment"] = (inflation_pivot["Typical Home Value as of 01/31/2024"] * 0.20).round(2)

# Compute the loan amount (principal) as Typical Home Value minus Down Payment
loan_amount = inflation_pivot["Typical Home Value as of 01/31/2024"] - inflation_pivot["Down Payment"]

# Column 2: Monthly Payment using the PMT formula for a 30-year loan which is amortized with fixed payments over that period
# PMT = (r * PV) / (1 - (1 + r)^(-n))
# where r = monthly interest rate, n = 360 months
inflation_pivot["Average Interest Rate (30 Yr Mortgage)"] = pd.to_numeric(
    inflation_pivot["Average Interest Rate (30 Yr Mortgage)"], errors="coerce"
)
monthly_rate = (inflation_pivot["Average Interest Rate (30 Yr Mortgage)"] / 100) / 12
n_periods = 360
inflation_pivot["Monthly Payment"] = (monthly_rate * loan_amount) / (1 - (1 + monthly_rate) ** (-n_periods))
inflation_pivot["Monthly Payment"] = inflation_pivot["Monthly Payment"].round(2)

# Column 3: Average Rent
# Read the average rent file which has "State" and "Average Rent" columns
df_rent = pd.read_excel("Home Price Index Recent Download/Average Rent by State.xlsx")
df_rent = df_rent[["State", "Average Rent"]]
# Merge the average rent data into merged_df on "State"
merged_df = pd.merge(inflation_pivot, df_rent, on="State", how="left")

# Column 4: Potential Monthly Payoff = Monthly Payment + Average Rent
merged_df["Potential Monthly Payoff"] = (merged_df["Average Rent"]-merged_df["Monthly Payment"]).round(2)

# Column 5: Average Rent Yield = (Potential Monthly Payoff / (Typical Home Value - Down Payment)) * 100
merged_df["Average Rent Yield"] = ((merged_df["Average Rent"] / loan_amount) * 100).round(2)

# Format the monetary columns as dollars (convert to string with dollar sign)
merged_df["Typical Home Value as of 01/31/2024"] = merged_df["Typical Home Value as of 01/31/2024"].apply(lambda x: f"${x:,.2f}")
merged_df["Down Payment"] = merged_df["Down Payment"].apply(lambda x: f"${x:,.2f}")

print("Added calculated columns")
merged_df

Added calculated columns


,State,2020-Q1,2020-Q2,2020-Q3,2020-Q4,2021-Q1,2021-Q2,2021-Q3,2021-Q4,2022-Q1,...,2024-Q4,Inflation 2024 %,Cumulative Inflation 2020-2024 %,Typical Home Value as of 01/31/2024,Average Interest Rate (30 Yr Mortgage),Down Payment,Monthly Payment,Average Rent,Potential Monthly Payoff,Average Rent Yield
0,Alabama,1.16,1.44,1.92,2.12,2.07,4.66,5.68,4.09,2.86,...,1.02,10.51,55.22,"$224,910.79",6.63,"$44,982.16",1152.70,1416.8,264.10,0.79
1,Alaska,1.20,0.92,1.27,0.84,1.09,3.73,4.54,1.45,2.63,...,-0.52,8.91,38.82,"$366,633.08",6.63,"$73,326.62",1879.04,1630.2,-248.84,0.56
2,Arizona,2.00,1.75,2.44,3.53,4.46,7.97,8.30,5.10,4.78,...,-0.14,8.36,64.21,"$424,841.80",6.63,"$84,968.36",2177.37,1732.5,-444.87,0.51
3,Arkansas,0.60,0.90,1.49,2.45,1.83,4.98,5.62,4.34,2.79,...,0.28,10.45,56.59,"$208,999.63",6.63,"$41,799.93",1071.15,1202.3,131.15,0.72
4,California,0.94,1.15,1.17,2.06,2.80,5.94,5.21,4.05,4.07,...,0.37,9.00,44.68,"$773,347.01",6.63,"$154,669.40",3963.51,2845.7,-1117.81,0.46
5,Colorado,1.09,1.18,1.58,2.05,2.69,7.19,5.93,2.84,4.14,...,-0.13,5.75,44.62,"$540,789.12",6.63,"$108,157.82",2771.62,2072.4,-699.22,0.48
6,Connecticut,0.38,1.05,2.22,2.72,2.62,5.14,5.42,2.05,2.70,...,1.01,18.16,63.03,"$404,161.82",6.63,"$80,832.36",2071.38,2248.4,177.02,0.70
7,Delaware,1.34,0.68,2.19,1.47,3.11,4.19,4.77,3.57,3.00,...,-0.13,11.74,53.17,"$380,840.16",6.63,"$76,168.03",1951.86,1810.6,-141.26,0.59
8,District of Columbia,0.86,0.22,1.17,2.20,1.06,4.19,2.38,0.88,2.50,...,-0.93,1.11,15.37,"$594,940.94",6.63,"$118,988.19",3049.15,2721.4,-327.75,0.57
9,Florida,1.53,1.27,1.74,2.67,3.25,6.55,7.39,5.63,5.65,...,0.94,8.74,70.92,"$385,540.77",6.63,"$77,108.15",1975.95,2150.5,174.55,0.70


In [22]:
# Save all results to "processed.xlsx" with three sheets
with pd.ExcelWriter("Real_Estate_Trends_USA.xlsx") as writer:
    # Sheet 1: Filtered data (raw)
    df_filtered.to_excel(writer, sheet_name="Raw_Data", index=False)
    
    # Sheet 2: Pivot of index_nsa
    df_pivot.to_excel(writer, sheet_name="HPI_2019_to_2024")
    
    # Sheet 3: All inflation data
    merged_df.to_excel(writer, sheet_name="inflation_data", index=False)

print("Data saved to 'processed.xlsx' with sheets: 'Filtered', 'Pivot', and 'inflation'.")


Data saved to 'processed.xlsx' with sheets: 'Filtered', 'Pivot', and 'inflation'.


In [23]:
'''
Appendix:
These are all the data sources I used to get the actual data

Sources Used:
Data is sourced from
https://www.fhfa.gov/data/hpi/datasets?tab=master-hpi-data
https://www.redfin.com/news/data-center/
https://www.freddiemac.com/pmms
https://www.zillow.com/research/data/
https://worldpopulationreview.com/state-rankings/average-rent-by-state

'''

'\nAppendix:\nThese are all the data sources I used to get the actual data\n\nSources Used:\nData is sourced from\nhttps://www.fhfa.gov/data/hpi/datasets?tab=master-hpi-data\nhttps://www.redfin.com/news/data-center/\nhttps://www.freddiemac.com/pmms\nhttps://www.zillow.com/research/data/\nhttps://worldpopulationreview.com/state-rankings/average-rent-by-state\n\n'